In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 800)

import scipy

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import detrend

import datetime as dt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

The KPN Group sells its products through different channels. 
* online 
* telephone 
* retail stores
* third-party stores. 

The KPN formula in the streets:
* KPN shop -- only KPN
* Telfort shop (KPN subsidiary) -- only Telfort
* Multibrand store -- combines the brands KPN, Telfort and XS4ALL,
* KPN XL store -- combines the consumer and small business segments.

## Research questions:
1. Which characteristics are most important for predicting the (contracted)
revenue per store formula (Telfort, KPN, KPN XL)?
2. Which Telfort stores should be transformed into KPN stores, and what is
the expected (contracted) revenue after this transformation?
3. Our budget enables the transformation of 5 KPN stores into KPN XL stores.
Which KPN stores should we transform to optimize the revenue?
4. Possibly continue with remaining questions to complete this advice.

## Evaluation
We find it more important that you are able to communicate and defend your advice well, than that you build the most advanced model. Therefore you should think about the complexity of the solution you choose in order to explain your decisions to an audience without explicit data science background. We do value a solid reasoning behind the chosen data transformations, etc.

Please send the two files before noon the day prior to the walkthrough and presentation sessions to Tom

## Case

In [66]:
case = pd.read_csv('dataset_shops.csv',
                   usecols = ['date', 'shop_id', 'committed_revenues_main_inc_vat', 'place', 
                              'shop_formula', 'dist_shop_closest_km', 'no_retail_shops', 
                              'no_companies', 'perc_owned_vs_rented_houses', 
                              'avg_value_household', 'avg_household_size', 'no_residents'])

# Other columns: tons of NA, not particularly relevant to question of interest (at first pass anyway)

case.date = pd.to_datetime(case.date)


In [65]:
case.columns
case.shape
case.head()
# case.info()

Index(['date', 'shop_id', 'committed_revenues_main_inc_vat', 'place',
       'shop_formula', 'dist_shop_closest_km', 'no_retail_shops',
       'no_companies', 'perc_owned_vs_rented_houses', 'avg_value_household',
       'avg_household_size', 'no_residents'],
      dtype='object')

(63256, 12)

,date,shop_id,committed_revenues_main_inc_vat,place,shop_formula,dist_shop_closest_km,no_retail_shops,no_companies,perc_owned_vs_rented_houses,avg_value_household,avg_household_size,no_residents
0,2017-01-02,2002001001,3852.0,eindhoven,Telfort,0.200938,4265.0,19605.0,45.0,211.0,1.9,226868.0
1,2017-01-03,2002001001,6768.0,eindhoven,Telfort,0.200938,4265.0,19605.0,45.0,211.0,1.9,226868.0
2,2017-01-04,2002001001,2664.0,eindhoven,Telfort,0.200938,4265.0,19605.0,45.0,211.0,1.9,226868.0
3,2017-01-05,2002001001,3672.0,eindhoven,Telfort,0.200938,4265.0,19605.0,45.0,211.0,1.9,226868.0
4,2017-01-06,2002001001,5328.0,eindhoven,Telfort,0.200938,4265.0,19605.0,45.0,211.0,1.9,226868.0


In [ ]:
for c in [case.columns]:
    if type(case[c][0]) != np.float64:
        print(c, len(case[c].unique()))

In [53]:
case.shape

for c in [case.columns]:
    print(case[c].isna().sum())


(63256, 25)

date                                   0
shop_id                                0
committed_revenues_main_inc_vat        0
sales_count_simonly                 4634
sales_count_alcatel                63185
sales_count_apple                  14754
sales_count_blackberry             63245
sales_count_doro                   62842
sales_count_emporia                62431
sales_count_fairphone              63207
sales_count_htc                    61603
sales_count_huawei                 54834
sales_count_lg                     60213
sales_count_nokia                  59053
sales_count_samsung                 9517
sales_count_xperia                 57322
place                                  0
shop_formula                           0
dist_shop_closest_km                   0
no_retail_shops                      899
no_companies                         899
perc_owned_vs_rented_houses          899
avg_value_household                 1849
avg_household_size                   899
no_residents    

In [47]:
case.shop_formula.unique()
case.place.unique()


array(['Telfort', 'KPN winkel', 'KPN XL winkel'], dtype=object)

array(['eindhoven', 'den bosch', 'arnhem', 'zwolle', 'rotterdam',
       'haarlem', 'alkmaar', 'amsterdam', 'den haag', 'groningen',
       'tilburg', 'apeldoorn', 'utrecht', 'almelo', 'enschede',
       'zoetermeer', 'almere', 'zaandam', 'hilversum', 'helmond',
       'nijmegen', 'maastricht', 'leeuwarden', 'breda', 'leiden',
       'doetinchem', 'veenendaal', 'amstelveen', 'purmerend',
       'middelburg', 'terneuzen', 'goes', 'roosendaal', 'bergen op zoom',
       'oosterhout', 'etten leur', 'alphen aan den rijn', 'ede',
       'oldenzaal', 'veldhoven', 'oss', 'heerhugowaard', 'oud-beijerland',
       'zwijndrecht', 'nieuwegein', 'sneek', 'assen', 'emmen',
       'winschoten', 'delft', 'rijswijk', 'leidschendam', 'naaldwijk',
       'hengelo', 'winterswijk', 'hoorn', 'den helder', 'beverwijk',
       'hoofddorp', 'weert', 'uden', 'waalwijk', 'drachten', 'heerenveen',
       'heerlen', 'roermond', 'sittard', 'venlo', 'beek', 'dordrecht',
       'gouda', 'spijkenisse', 'vlaardingen', 